## Ejercicio - Quién está hablando - nlp

In [3]:
import numpy as np
import pandas as pd
import nltk
import string
import seaborn as sns
import matplotlib.pyplot as plt 

In [6]:
from nltk.corpus import stopwords
from unidecode import unidecode

* Descargo los stopwords que usare en el analisis de texto

In [7]:
nltk.download("stopwords")

[nltk_data] Downloading package stopwords to
[nltk_data]     C:\Users\Anna\AppData\Roaming\nltk_data...
[nltk_data]   Unzipping corpora\stopwords.zip.


True